# Assignment 3

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
import numpy as np

#### Data Preparation

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
unlabeled_set, labeled_set = torch.utils.data.random_split(trainset, [50000, 10000])

trainloader = torch.utils.data.DataLoader(labeled_set, batch_size=64, shuffle=True)


testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

#### CNN Model 

In [3]:
class CNN(nn.Module):
    def __init__(self):

        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)

        self.fc2 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))

        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64 * 7 * 7)

        x = F.relu(self.fc1(x))

        x = self.dropout(x)
        
        x = self.fc2(x)
        return x

In [4]:
model = CNN().to("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

#### Active Learning Integration

In [5]:
def calculate_uncertainty(outputs):

    probs = F.softmax(outputs, dim=1)

    least_confidence = 1 - probs.max(dim=1).values

    prediction_entropy = -(probs * probs.log()).sum(dim=1)

    margin_sampling = probs.topk(2, dim=1).values[:, 0] - probs.topk(2, dim=1).values[:, 1]

    return least_confidence, prediction_entropy, margin_sampling

def calculate_diversity(features):

    features_np = features.cpu().detach().numpy()

    cosine_similarity = 1 - pairwise_distances(features_np, metric='cosine')

    l2_norm = pairwise_distances(features_np, metric='euclidean')
    
    return cosine_similarity.mean(), l2_norm.mean()

##### Train and Evaluate

In [6]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100.0 * correct / total
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%")
    scheduler.step()

model.eval()
correct = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to("cuda" if torch.cuda.is_available() else "cpu"), labels.to("cuda" if torch.cuda.is_available() else "cpu")
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()

print(f"Test Accuracy: {100.0 * correct / len(testset):.2f}%")

Epoch 1, Loss: 0.9098, Accuracy: 66.81%
Epoch 2, Loss: 0.5568, Accuracy: 79.67%
Epoch 3, Loss: 0.4829, Accuracy: 82.35%
Epoch 4, Loss: 0.4351, Accuracy: 83.98%
Epoch 5, Loss: 0.3989, Accuracy: 85.63%
Epoch 6, Loss: 0.3463, Accuracy: 87.75%
Epoch 7, Loss: 0.3271, Accuracy: 88.16%
Epoch 8, Loss: 0.3118, Accuracy: 88.53%
Epoch 9, Loss: 0.3008, Accuracy: 89.22%
Epoch 10, Loss: 0.2797, Accuracy: 89.79%
Test Accuracy: 87.90%
